#install

In [ ]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
!apt install python3-rtree 
# Install Geopandas
!pip install git+git://github.com/geopandas/geopandas.git
# Install descartes - Geopandas requirment
!pip install descartes 
# Install Folium for Geographic data visualization
!pip install folium
# Install plotlyExpress
!pip install plotly_express
!pip install Shapely
!pip install plotly_express

Reading package lists... Done
Building dependency tree       
Reading state information... Done
gdal-bin is already the newest version (2.2.3+dfsg-2).
python-gdal is already the newest version (2.2.3+dfsg-2).
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  python-numpy-doc python3-nose python3-numpy-dbg
The following NEW packages will be installed:
  python3-gdal python3-numpy
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,288 kB of archives.
After this operation, 13.2 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 python3-numpy amd64 1:1.13.3-2ubuntu1 [1,943 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python3-gdal amd64 2.2.3+dfsg-2 [346 kB]
Fetched 2,288 kB in 1s (2,388 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 155047 files and directories currently installed.)
Preparing to unpack .../python3-numpy_

In [ ]:
from shapely.geometry import Point,Polygon
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt 
import folium
from folium import plugins
from folium.plugins import HeatMap
import plotly_express as px
from google.colab import drive
from IPython.display import display
import glob
import re
import plotly_express as px
import seaborn as sns
import os
from os.path import exists
drive.mount('/content/drive')

Mounted at /content/drive


#Read

In [ ]:
!ls '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips'

lebeled		     trips_2019_2_10.csv  trips_2019_3_13.csv
trips_2019_1_10.csv  trips_2019_2_11.csv  trips_2019_3_14.csv
trips_2019_1_11.csv  trips_2019_2_12.csv  trips_2019_3_15.csv
trips_2019_1_12.csv  trips_2019_2_13.csv  trips_2019_3_16.csv
trips_2019_1_13.csv  trips_2019_2_14.csv  trips_2019_3_17.csv
trips_2019_1_14.csv  trips_2019_2_15.csv  trips_2019_3_18.csv
trips_2019_1_15.csv  trips_2019_2_16.csv  trips_2019_3_19.csv
trips_2019_1_16.csv  trips_2019_2_17.csv  trips_2019_3_1.csv
trips_2019_1_17.csv  trips_2019_2_18.csv  trips_2019_3_20.csv
trips_2019_1_18.csv  trips_2019_2_19.csv  trips_2019_3_21.csv
trips_2019_1_19.csv  trips_2019_2_1.csv   trips_2019_3_22.csv
trips_2019_1_1.csv   trips_2019_2_20.csv  trips_2019_3_23.csv
trips_2019_1_21.csv  trips_2019_2_21.csv  trips_2019_3_24.csv
trips_2019_1_22.csv  trips_2019_2_22.csv  trips_2019_3_25.csv
trips_2019_1_23.csv  trips_2019_2_23.csv  trips_2019_3_26.csv
trips_2019_1_24.csv  trips_2019_2_24.csv  trips_2019_3_27.csv
trips_2019_1_25.

In [ ]:
tripPath = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/trips_and_stops_NE_2019'
districtPath = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/District_NE.geojson'

In [ ]:
district_NE = gpd.read_file(districtPath)
district_NE

,AMP_CODE,PRV_CODE,AMP_NAME_T,AMP_NAME_E,Shape_Leng,Shape_Area,geometry
0,3001,30,เมืองนครราชสีมา,Mueang Nakhon Ratchasima,160600.306403,8.856562e+08,"POLYGON ((102.18341 15.13798, 102.18298 15.137..."
1,3002,30,ครบุรี,Khon Buri,261136.836300,2.168627e+09,"POLYGON ((102.15081 14.61461, 102.15115 14.614..."
2,3003,30,เสิงสาง,Soeng Sang,181829.710406,1.097496e+09,"POLYGON ((102.49675 14.50541, 102.49686 14.505..."
3,3004,30,คง,Khong,208115.123225,7.534105e+08,"POLYGON ((102.27233 15.53663, 102.27538 15.536..."
4,3005,30,บ้านเหลื่อม,Ban Lueam,88918.433064,2.569881e+08,"POLYGON ((102.12032 15.64497, 102.12072 15.644..."
...,...,...,...,...,...,...,...
316,3902,39,นากลาง,Na Klang,190622.272867,7.934911e+08,"POLYGON ((102.13153 17.49852, 102.13277 17.498..."
317,3903,39,โนนสัง,Non Sang,127457.559383,7.620915e+08,"POLYGON ((102.62819 17.01421, 102.62874 17.013..."
318,3904,39,ศรีบุญเรือง,Si Bun Rueang,253122.478708,1.053086e+09,"POLYGON ((102.06074 17.20408, 102.06092 17.204..."
319,3905,39,สุวรรณคูหา,Suwannakhuha,142751.437069,6.590840e+08,"POLYGON ((102.31839 17.67681, 102.31874 17.667..."


#Algorithm

In [ ]:
def check_district(lon,lat):
  data = zip(district_NE.geometry,district_NE.AMP_NAME_T)
  point = Point(lon,lat)
  for index in data:
    if point.within(index[0]):
      return index[1]
  return False

In [ ]:
check_district(102.151459,14.915597)

'เมืองนครราชสีมา'

In [ ]:
def vehicle_group(df):
   df['veh_group']=df['unit_type'].apply(lambda x: 0 if x == 3 else x)
   df['veh_group']=df['unit_type'].apply(lambda x: 1 if x == 5 or x == 8 or x == 9 else x)
   df['veh_group']=df['unit_type'].apply(lambda x: 2 if x == 6 or x == 7 else x)

In [ ]:
def days_group(df):
  df.time_stamp = df.time_stamp.astype("datetime64")
  if df.loc[0].time_stamp.day_name() == 'Monday':
    df['day_group'] = 0
  if (df.loc[0].time_stamp.day_name() == 'Tuesday') or (df.loc[0].time_stamp.day_name() == 'Wednesday') or (df.loc[0].time_stamp.day_name() == 'Thursday'):
    df['day_group'] = 1
  if df.loc[0].time_stamp.day_name() == 'Friday':
    df['day_group'] = 2
  if df.loc[0].time_stamp.day_name() == 'Saturday' or (df.loc[0].time_stamp.day_name() == 'Sunday'):
    df['day_group'] = 3
  # df['day_group']=df['time_stamp'].apply(lambda x: 0 if x.day_name() == 'Monday' else x)
  # df['day_group']=df['time_stamp'].apply(lambda x: 1 if x.day_name() == 'Tuesday' or x.day_name() == 'Wednesday' or x.day_name() == 'Thursday' else x)
  # df['day_group']=df['time_stamp'].apply(lambda x: 2 if x.day_name() == 'Friday' else x)
  # df['day_group']=df['time_stamp'].apply(lambda x: 3 if x.day_name() == 'Saturday' or x.day_name() == 'Sunday'else x)

#Feature

In [ ]:
Path = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/Feature_trips'
days = range(1,32)
month = 3
for day in days:
    export_filename = Path + f'/trips_2019_{month}_{day}.csv'

      #### check file exists
    if os.path.exists(export_filename):
      print(f'trips_2019_{month}_{day}.csv already taken!!')
      continue
    else:
      with open(export_filename, 'w') as fp:
        pass
      if os.path.exists(export_filename):
        print(f'trips_2019_{month}_{day}.csv creating...')
        pass

    filenames = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/' + f'trips_2019_{month}_{day}.csv'
    trip = pd.read_csv(filenames)

    vehicle_group(trip)
    days_group(trip)

    trip.to_csv(export_filename, encoding='utf-8', index = False)
    print(f'trips {month} {day} done')


trips_2019_3_1.csv creating...
trips 3 1 done
trips_2019_3_2.csv creating...
trips 3 2 done
trips_2019_3_3.csv creating...
trips 3 3 done
trips_2019_3_4.csv creating...
trips 3 4 done
trips_2019_3_5.csv creating...
trips 3 5 done
trips_2019_3_6.csv creating...
trips 3 6 done
trips_2019_3_7.csv creating...
trips 3 7 done
trips_2019_3_8.csv creating...
trips 3 8 done
trips_2019_3_9.csv creating...
trips 3 9 done
trips_2019_3_10.csv creating...
trips 3 10 done
trips_2019_3_11.csv creating...
trips 3 11 done
trips_2019_3_12.csv creating...
trips 3 12 done
trips_2019_3_13.csv creating...
trips 3 13 done
trips_2019_3_14.csv creating...
trips 3 14 done
trips_2019_3_15.csv creating...
trips 3 15 done
trips_2019_3_16.csv creating...
trips 3 16 done
trips_2019_3_17.csv creating...
trips 3 17 done
trips_2019_3_18.csv creating...
trips 3 18 done
trips_2019_3_19.csv creating...
trips 3 19 done
trips_2019_3_20.csv creating...
trips 3 20 done
trips_2019_3_21.csv creating...
trips 3 21 done
trips_2019

##trip

In [ ]:
day = 5
month = 2

In [ ]:
filenames = '/content/drive/My Drive/2021 Route Prediction/Project-1/Source-Code/data/3.1_cluster_on_trips/' + f'trips_2019_{month}_{day}.csv'
trip = pd.read_csv(filenames)

##vehicle type

In [ ]:
trip

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,cluster
0,2019-02-05 00:00:05,067000700000000001000016154,16.525710,102.831300,59,7.0,0,-1
1,2019-02-05 00:00:15,067000700000000001000016154,16.527160,102.831000,60,7.0,0,-1
2,2019-02-05 00:00:25,067000700000000001000016154,16.528620,102.830690,60,7.0,0,-1
3,2019-02-05 00:00:55,067000700000000001000016154,16.533100,102.829830,62,7.0,0,-1
4,2019-02-05 00:01:05,067000700000000001000016154,16.534670,102.830000,64,7.0,0,-1
...,...,...,...,...,...,...,...,...
712496,2019-02-05 07:43:11,0390002000000000000EE100465,14.988145,102.644621,0,7.0,9256,-1
712497,2019-02-05 07:47:30,0390002000000000000EE100465,14.989004,102.652895,22,7.0,9256,-1
712498,2019-02-05 07:52:24,0390002000000000000EE100465,14.992604,102.691226,63,7.0,9256,-1
712499,2019-02-05 07:58:51,0390002000000000000EE100465,14.990155,102.705568,27,7.0,9256,-1


In [ ]:
vehicle_group(trip)
trip

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,veh_group
0,2019-02-05 00:00:05,067000700000000001000016154,16.525710,102.831300,59,7.0,0,2.0
1,2019-02-05 00:00:15,067000700000000001000016154,16.527160,102.831000,60,7.0,0,2.0
2,2019-02-05 00:00:25,067000700000000001000016154,16.528620,102.830690,60,7.0,0,2.0
3,2019-02-05 00:00:55,067000700000000001000016154,16.533100,102.829830,62,7.0,0,2.0
4,2019-02-05 00:01:05,067000700000000001000016154,16.534670,102.830000,64,7.0,0,2.0
...,...,...,...,...,...,...,...,...
712496,2019-02-05 07:43:11,0390002000000000000EE100465,14.988145,102.644621,0,7.0,9256,2.0
712497,2019-02-05 07:47:30,0390002000000000000EE100465,14.989004,102.652895,22,7.0,9256,2.0
712498,2019-02-05 07:52:24,0390002000000000000EE100465,14.992604,102.691226,63,7.0,9256,2.0
712499,2019-02-05 07:58:51,0390002000000000000EE100465,14.990155,102.705568,27,7.0,9256,2.0


In [ ]:
days_group(trip)
trip

,time_stamp,unit_id,lat,lon,speed,unit_type,trip_id,veh_group,day
0,2019-02-05 00:00:05,067000700000000001000016154,16.525710,102.831300,59,7.0,0,2.0,1
1,2019-02-05 00:00:15,067000700000000001000016154,16.527160,102.831000,60,7.0,0,2.0,1
2,2019-02-05 00:00:25,067000700000000001000016154,16.528620,102.830690,60,7.0,0,2.0,1
3,2019-02-05 00:00:55,067000700000000001000016154,16.533100,102.829830,62,7.0,0,2.0,1
4,2019-02-05 00:01:05,067000700000000001000016154,16.534670,102.830000,64,7.0,0,2.0,1
...,...,...,...,...,...,...,...,...,...
712496,2019-02-05 07:43:11,0390002000000000000EE100465,14.988145,102.644621,0,7.0,9256,2.0,1
712497,2019-02-05 07:47:30,0390002000000000000EE100465,14.989004,102.652895,22,7.0,9256,2.0,1
712498,2019-02-05 07:52:24,0390002000000000000EE100465,14.992604,102.691226,63,7.0,9256,2.0,1
712499,2019-02-05 07:58:51,0390002000000000000EE100465,14.990155,102.705568,27,7.0,9256,2.0,1
